In [1]:
from google.colab import userdata

In [2]:
!pip install flask-ngrok
!pip install pyngrok
!pip install deepforest

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.0 MB/s eta 0:00:00


In [3]:
!ngrok authtoken '2v2XB03eg5M6UL8bGB4pQ5bSrgg_4p1bzmVYu3RYXFrK7rUQj'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
from deepforest import visualize
from deepforest import main
from deepforest.model import CropModel
from torchvision import transforms
import os, base64
import pandas as pd
from io import BytesIO
from PIL import Image
import numpy as np

In [5]:
deepforest_model = main.deepforest(config_args={"workers": 4})
deepforest_model.use_release()

Reading config file: /usr/local/lib/python3.11/dist-packages/deepforest/data/deepforest_config.yml
setting config workers to 4


Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth
100%|██████████| 130M/130M [00:00<00:00, 186MB/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Reading config file: /usr/local/lib/python3.11/dist-packages/deepforest/data/deepforest_config.yml


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


model.safetensors:   0%|          | 0.00/129M [00:00<?, ?B/s]

In [9]:
def get_transform_tree(self, augment):
        data_transforms = []
        data_transforms.append(transforms.Resize(256))
        data_transforms.append(transforms.CenterCrop(224))
        data_transforms.append(transforms.ToTensor())
        data_transforms.append(transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))
        if augment:
            data_transforms.append(transforms.RandomHorizontalFlip(0.5))
        return transforms.Compose(data_transforms)
crop_model = CropModel.load_from_checkpoint('/content/drive/MyDrive/MMF-Ngrok/indian_trees_resnet50_lightning_01.ckpt')
crop_model.get_transform = get_transform_tree

def annotate_image(image, boxes_df):
    annotated = visualize.plot_predictions(np.array(image), boxes_df, color=(0, 165, 255))
    annotated = Image.fromarray(annotated, 'RGB')
    buffered = BytesIO()
    annotated.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def calculate_green_cover(boxes_df, original_width, original_height):
    boxes_df["box_area"] = (boxes_df["xmax"] - boxes_df["xmin"]) * (boxes_df["ymax"] - boxes_df["ymin"])
    total_box_area = boxes_df["box_area"].sum()
    image_area = original_width * original_height
    return round((total_box_area / image_area) * 100, 4)

def predict_trees_df(model, image, patch_size, path):
    global crop_model
    return model.predict_tile(path, patch_size=patch_size, crop_model=crop_model)

def get_tree_classes(prediction):
    df = pd.DataFrame(prediction)
    df['cropmodel_name'] = df['cropmodel_label'].replace({i: key for i, key in enumerate(['amla', 'asopalav', 'babul', 'bamboo', 'banyan', 'bili', 'cactus', 'champa', 'coconut', 'garmalo', 'gulmohor', 'gunda', 'jamun', 'kanchan', 'kesudo', 'khajur', 'mango', 'motichanoti', 'neem', 'nilgiri', 'other', 'pilikaren', 'pipal', 'saptaparni', 'shirish', 'simlo', 'sitafal', 'sonmahor', 'sugarcane', 'vad'])})
    df_cleaned = df[~df.isin(["other"]).any(axis=1)]
    top_n_classes = df_cleaned.nlargest(int(df_cleaned.shape[0] // 2), 'cropmodel_score')
    names = dict(top_n_classes[['cropmodel_name']].value_counts())
    return {name[0]: int(names[name]) for name in names}

def load_image(image_base64):
    img = Image.open(BytesIO(base64.b64decode(image_base64)))
    img.save('buffer/pred_image.png')

    return img, 'buffer/pred_image.png'

def predict_on_chunk(model, image_base64_chunk, patch_size):
    predictions = []
    for image_base64 in image_base64_chunk:
        image, path = load_image(image_base64)
        prediction = predict_trees_df(model, image, patch_size, path)
        classification = get_tree_classes(prediction)
        annotated = annotate_image(image, prediction)
        green_cover = calculate_green_cover(prediction, image.width, image.height)
        predictions.append({"url": annotated, "count": len(prediction), "percentage": green_cover, "classifications": classification})
    return predictions

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 195MB/s]


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
from flask import Flask, request
app = Flask(__name__)

@app.route("/")
def home():
  return "MammyForest"

@app.post("/predict-trees")
def predict():
  data = request.get_json()
  image_base64_chunk = data["image_base64_chunk"]
  patch_size = data["patch_size"]
  predictions = predict_on_chunk(deepforest_model, image_base64_chunk, patch_size)
  return predictions

from pyngrok import ngrok

public_url = ngrok.connect(5000)

print("Running at: ", public_url)

app.run()

Running at:  NgrokTunnel: "https://28e9-35-247-136-105.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on /predict-trees [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-retu

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


29 predictions in overlapping windows, applying non-max supression
29 predictions kept after non-max suppression


...

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


47 predictions in overlapping windows, applying non-max supression
47 predictions kept after non-max suppression


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [27/Apr/2025 19:25:29] "POST /predict-trees HTTP/1.1" 200 -
